## 准备工作

### 设置参数

In [ ]:
# 下载过程中,除非打印提示要求,请尽量不要手动操作脚本唤起的浏览器

# 请在下方双引号中填充 WoS 检索表达式
# 检索表达式将填充在高级搜索模式界面下检索式预览框,举个例子:
# "SO=(Frontiers in Psychology)" 意思严格匹配文献来源刊物名称
# 为 Frontiers in Psychology，不区分大小写
query_content = "SO=(Water Research)".upper()

# 请在下方双引号中填充自动化下载浏览器实例用户数据存储文件夹路径
# 实例可以理解为应用多开,最好指定一个空文件夹路径
# 亦可指定一个不存在的文件夹,代码将自动为您创建
# 不要用本文档代码同时创建多个浏览器实例并指定同一个配置文
# 初次初始化实例成功后,建议备份一份览器实例用户数据,暂用于后续解决长时间实例化失败问题修复
# 结束某次自动化任务，最好执行本文档底部的 “干净退出本次自动化下载浏览器实例“
# 在 Windows 上路径可能长得不一样，不用调整
# 所有路径都需提供绝对路径

user_data_path = ""

# WoS 引文导出下载路径
wos_download_path = ""

# 下载任务日志
download_task_log_path = ""

# 请输入你的日光图书馆帐号
usrname = ""

# 请输入你的日光图书馆密码
password = ""

### 导入预定义函数

In [ ]:
from bricklayer import aggregate_failed_records
from bricklayer import create_chrome_driver
from bricklayer import extract_content
from bricklayer import find_and_move_files
from bricklayer import get_power_law_pause
from bricklayer import goto_wos_captcha
from bricklayer import hover_pause_click
from bricklayer import human_clear
from bricklayer import human_type
from bricklayer import initiate_sunshine_login
from bricklayer import is_domain_present
from bricklayer import is_new_tab_opened
from bricklayer import is_sunshine_logged_in
from bricklayer import login_to_channel
from bricklayer import print_verbose
from bricklayer import random_delay
from bricklayer import random_scroll_partial
from bricklayer import save_log
from bricklayer import scroll_to_top
from bricklayer import switch_tab_by_domain
from bricklayer import validate_and_create_path
from bricklayer import verify_element_value
from bricklayer import wos_advanced_search
from bricklayer import wos_close_pendo
from bricklayer import wos_detective_inspector
from bricklayer import wos_download_bricklayer
from bricklayer import wos_download_contractor
from bricklayer import wos_export_dialog_exists
from bricklayer import wos_get_download_ranges
from bricklayer import wos_goto_advanced_search
from bricklayer import wos_log_query_task
from bricklayer import wos_query_result_arrange
from bricklayer import wos_query_result_count
from bricklayer import wos_switch_english

## 半自动化下载引文

In [ ]:
# 校验路径
validate_and_create_path(user_data_path)
validate_and_create_path(wos_download_path)
validate_and_create_path(download_task_log_path)

In [ ]:
# 初始化自动化测试浏览器实例，若存在历史自动化浏览器用户数据
# 则利用其恢复权限通道登陆状态，以避免权限通道方误判帐号借用而封号
driver = login_to_channel(
  channel="sunshine",
  username=usrname,
  password=password,
  user_data_path=user_data_path,
  wos_download_path=wos_download_path)

In [ ]:
# 导航到进入 Web of Science 需要的人机验证
# 如失败,请重行执行本代码块
# 待优化,要避免人工不小心干扰就失败
goto_wos_captcha(driver)

In [ ]:
# 切换到 Web of Science 基础查询页面
# 用户手动 Tab 切换到代码点击链接产生的跳转,并不会作用到实例
# 若想后续代码操作基于当前 Tab,必须用代码切换
switch_tab_by_domain(driver=driver, domain_type="wos")

In [ ]:
# 参考注释选择性执行，首次执行一次后续应该不用执行

# 若有 Cookie 询问弹窗，运行本代码块关闭
# 亦可手动关闭弹窗，但不要手动接受
wos_close_pendo(driver)

In [ ]:
# 切换语言为英语以重用他人代码
# 待完善提示
wos_switch_english(driver)

In [ ]:
# 跳转到高级查询界面
wos_goto_advanced_search(driver)

In [ ]:
# 参考注释选择性执行

# 如有弹窗《Are you getting the most out of the Web of Science?》
# 则执行本代码块关闭，亦可手动关闭弹窗
wos_close_pendo(driver)

In [ ]:
# 填写查询内容，执行高级查询
wos_advanced_search(driver, query=query_content)

In [ ]:
# 从旧到新排序查询结果，以便后续增量更新，并确保后续查漏补缺下载时记录顺序一致性。
wos_query_result_arrange(driver)

In [ ]:
# 通过若干次“下”键，以留出导出内容类型选项显示空间
random_scroll_partial(driver, ['down'], arrows=9)

In [ ]:
# 批量下载查询结果
# 下载过程中，不要手动滚动页面，否则会定位不准
# 此处调用但下载完整性校验还未充分测试
wos_download_contractor(
  driver=driver,
  start=1,
  query_content=query_content,
  download_task_log_path=download_task_log_path,
  wos_download_path=wos_download_path)

### 校验、迁移下载文件夹以归档下载任务

In [ ]:
# 查看是否有失败记录
aggregate_failed_records(
  directory=download_task_log_path,
  query_content=query_content,
  file_prefix="task_details_")

In [ ]:
# 查找下载路径中指定期刊的引文和日志文件，并转移到指定路径
# 只支持 "SO=(Journal of Hazardous Materials)" 形式的期刊名称提取
journal_name = extract_content(query_content) 
find_and_move_files(
  source_path=wos_download_path,
  target_path="",
  journal_name=journal_name)

### 干净退出本次自动化下载浏览器实例

In [15]:
driver.quit()